In [ ]:
%pip install matplotlib numpy pycryptodome

In [ ]:
target_size_bytes= 1 * 1024 * 1024 * 1024  # 1GB
filename = 'large_file.txt'

line_content = '1' * 60 + '\n'  # 61 bytes mỗi dòng (60 ký tự + newline)
line_bytes = len(line_content.encode('utf-8'))  # Đảm bảo theo UTF-8

total_lines = target_size_bytes // line_bytes  # Số dòng cần tạo

with open(filename, 'w', encoding='utf-8') as f:
    for _ in range(total_lines):
        f.write(line_content)

print(f"File '{filename}' đã được tạo với {total_lines} dòng (~1GB).")

In [ ]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad

In [ ]:
data = b'0123456789abcdef'
while len(data) < AES.block_size:
    data += data[-16]

print(AES.block_size)
print(data)
print(pad(data, AES.block_size))

In [ ]:
KEY_128 = 16
KEY_192 = 24
KEY_256 = 32

key = get_random_bytes(KEY_128)

key.hex()

In [ ]:
cipher_cbc = AES.new(key, AES.MODE_CBC)
iv = cipher_cbc.iv
iv.hex()

### test `pycryptodome` library

#### AES (Rijndael)

use mode CTR, reasons:
- fast
- stream friendly, no padding
- no need AEAD for pure benchmarking

TODO:
-  maybe test more with GCM mode

Test with file 1GB

In [ ]:
import timeit, os

In [ ]:
use_aesni = True
chunk_MB = 100
cipher_file = AES.new(key, AES.MODE_CTR, use_aesni=use_aesni)
nonce_file = cipher_file.nonce
# assert len(nonce_file) == 16

if os.path.exists('large_file_encrypted.bin'):
    os.remove('large_file_encrypted.bin')

with open('large_file.txt', 'rb') as f, open('large_file_encrypted.bin', 'wb') as enc_f:
    t1 = timeit.default_timer()
    while file_data := f.read(chunk_MB * 1024 * 1024):
        enc_f.write(cipher_file.encrypt(file_data))
    t2 = timeit.default_timer()

tag = cipher_file.digest()
print(f'Use AES-NI: {use_aesni}')
print(f'Chunk size: {chunk_MB} MB')
print(f"Nonce: {nonce_file.hex()}")
print(f"Tag: {tag.hex()}")
print(f"Encryption Time: {t2 - t1} seconds")
print(f'Throughput = {os.path.getsize("large_file_encrypted.bin") / (t2 - t1) / (1024 * 1024):.2f} MB/s')

In [ ]:
decipher_file = AES.new(key, AES.MODE_CTR, nonce=nonce_file)

with open('large_file_encrypted.bin', 'rb') as enc_f, open(os.devnull, 'wb') as dec_f:
    t1 = timeit.default_timer()
    while enc_data := enc_f.read(1024 * 1024):  # 1MB
        dec_f.write(decipher_file.decrypt(enc_data))
    t2 = timeit.default_timer()

# try:
#     decipher_file.verify(tag)
#     print("Decryption successful, tag verified.")
# except ValueError:
#     print("Decryption failed, tag verification error.")

print(f"Decryption Time: {t2 - t1} seconds")

test in memory

In [ ]:
use_aesni = True
cipher_in_mem = AES.new(key, AES.MODE_CTR, use_aesni=use_aesni)

data = b"\x01" * (1024 * 1024 * 1024)

exec_time = timeit.timeit(lambda: cipher_in_mem.encrypt(data), number=1)

print(f'Use AES-NI: {use_aesni}')
print(f"Nonce: {cipher_in_mem.nonce.hex()}")
# print(f"Tag: {cipher_in_mem.tag.hex()}")
print(f"Encryption Time: {exec_time} seconds")
print(f'Throughput = {len(data) / (exec_time) / (1024 * 1024):.2f} MB/s')

### test `cryptography` lib

In [ ]:
%pip install cryptography

in memory

In [3]:
def to_MBPS(seconds: float | list[float]) -> float | list[float]:
    if isinstance(seconds, list):
        return [1024 / sec for sec in seconds]
    return 1024 / seconds

In [4]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import timeit

key = b"\x00" * 16
iv = b"\x00" * 16
data = b"\x00" * (1024 * 1024 * 1024)

cipher2 = Cipher(algorithms.AES(key), modes.CTR(iv), backend=default_backend())
encryptor = cipher2.encryptor()

# t1 = timeit.default_timer()
exec_time = timeit.repeat(lambda: encryptor.update(data), repeat=10, number=1)
# t2 = timeit.default_timer()

print(f"Encryption Time: {exec_time}")
print(f"Throughput MB/s: {to_MBPS(exec_time)}")


Encryption Time: [0.450697250002122, 0.40014326000164147, 0.4611077640001895, 0.42400942399763153, 0.41438340500099, 0.41511087999970187, 0.4346058589981112, 0.4082150129979709, 0.42415977000200655, 0.4190370249998523]
Throughput MB/s: [2272.0351632835095, 2559.0834642467785, 2220.7390114549867, 2415.04066193991, 2471.1414299941707, 2466.8107952283385, 2356.1578354249714, 2508.4819700276184, 2414.18463612227, 2443.6981433809124]
